To have functionality you need to have the following libraries installed 

```python
import matplotlib.animation
from IPython.display import HTML 
import numpy
import matplotlib.pyplot
import seaborn
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns 
sns.set_context("notebook", font_scale=2, rc={"lines.linewidth": 2.5})

In [3]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

---
---
---

So to do the coupling we are simulating two straight lines to a certain point. Then at the next points they will couple, and then the run continues from there. 

What we essentially need to do is run two loops of the 1d FDTD code that stops in the coupling section to preform the coupling and then continues on the path of simulating the rest. 

So the structure of operation is done as follows.

1. Run first section of the main lead
2. Run coupling
3. Run ring
4. Run second section of main lead
5. Repeat over time

---

To do the run function we need the following. 

1. The length of first part of run
2. The size of loop
3. The coupling
4. The loss in loop
5. The length of second part of run
6. The imp0
7. The maxTime for the run

- We start by defining the arrays for each section of fields that will be processed.

- We then need to execute the initial part of the run with input.
- Next we need to run the full range of the first part which we do in a separate function

- We then need to preform the coupling computation which we also do in a separate function

In [43]:
def run (size1, size2, loopSize, omega = 1, coupling = 1, loopLoss = 0, imp0 = 377.0, maxTime = 250):
    ez1 = np.zeros((maxTime, size1))
    hy1 = np.zeros((maxTime, size1))

    ezLoop = np.zeros((maxTime, loopSize))
    hyLoop = np.zeros((maxTime, loopSize))

    ez2 = np.zeros((maxTime, size2))
    hy2 = np.zeros((maxTime, size2))

    for qTime in range(0, maxTime, 1):
        

        ez1[qTime], hy1[qTime] = straightSection(ez1[qTime-1], hy1[qTime-1], size1)

        ezLoop[qTime-1][0], hyLoop[qTime-1][0], ez2[qTime-1][0], hy2[qTime-1][0] = couple(ezLoop[qTime-1][-1], hyLoop[qTime-1][-1], ez1[qTime][-1], hy1[qTime][-1], coupling)

        # (ezIniLoop, hyIniLoop, ezIniRun, hyIniRun, coupling=1)
        # return ezLoop, hyLoop, ezContinues, hyContinues
        

        #FOR TEST RUN NO LOSS IS ASSUMED IN LOOP

        ezLoop[qTime], hyLoop[qTime] = straightSection(ezLoop[qTime-1], hyLoop[qTime-1], loopSize)

        ez2[qTime], hy2[qTime] = straightSection(ez2[qTime-1], hy2[qTime-1], size2)

        ez1[qTime][0] = np.exp(1j*omega*maxTime)

    return ez1, hy1, ez2, hy2, ezLoop, hyLoop

The function to run any of the straight lines is just running the hy and ez across it once.

- The inputs need to be ez and hy for a given time, as well as the optional imp0.

In [34]:
def straightSection (ez, hy, size, imp0 = 377.0):

    for mm in range(0, size-1, 1):
        hy[mm] = hy[mm] + (ez[mm + 1] - ez[mm]) / imp0

    for mm in range(1, size, 1):
        ez[mm] = ez[mm] + (hy[mm] - hy[mm - 1]) * imp0

    return ez, hy

The function to preform coupling takes in values for ez and hy values for the loop and for the run, applies coupling to them, and then returns how much ez and hy goes in the loop and how much ez and hy continues straight. 

For the $e^{i\omega t}$

the $\omega$ is the frequency where the wavelength at $\omega = 1$ the $\lambda = 2\pi$

In [24]:
def couple (ezIniLoop, hyIniLoop, ezIniRun, hyIniRun, coupling=1):

    ezLoop, ezContinues = coupleEach(ezIniLoop, ezIniRun, coupling)
    hyLoop, hyContinues = coupleEach(hyIniLoop, hyIniRun, coupling)

    return ezLoop, hyLoop, ezContinues, hyContinues

In [25]:
def coupleEach(in1, in2, coupling):
    matrix = np.zeros((2,2), dtype=complex)

    e_Apow = np.power(np.e,1j*-1*coupling)
    e_Bpow = np.power(np.e,1j*coupling)

    matrix[0,0] = in1
    matrix[0,1] = e_Apow
    matrix[1,0] = e_Bpow
    matrix[1,1] = in2

    eig = np.linalg.eigvals(matrix)

    return eig[0].real, eig[1].real

In [26]:
a = 2
v=0.8
w=1

freq = 1*np.pi/a
omega_0 = 0
matrix = np.zeros((2,2), dtype=complex)


e_Apow = np.power(np.e,1j*-1*freq*a)
e_Bpow = np.power(np.e,1j*freq*a)

matrix[0,0] = omega_0
matrix[0,1] = 1
matrix[1,0] = 1
matrix[1,1] = omega_0

eig = np.linalg.eigvals(matrix)

In [27]:
eig[0].real

0.9999999999999996

In [28]:
def runh (size, imp0 = 377.0, maxTime = 250):
    ez = np.zeros((maxTime, size))
    hy = np.zeros((maxTime, size))


    for qTime in range(0, maxTime, 1):
        for mm in range(0, size-1, 1):
            hy[qTime][mm] = hy[qTime-1][mm] + (ez[qTime-1][mm + 1] - ez[qTime-1][mm]) / imp0

        for mm in range(1, size, 1):
            ez[qTime][mm] = ez[qTime-1][mm] + (hy[qTime][mm] - hy[qTime][mm - 1]) * imp0

        ez[qTime][0] = np.exp(-(qTime-30)*(qTime-30)/100)

    return hy, ez

In [29]:
def part1Run (size, imp0 = 377.0, maxTime = 250):
    ez = np.zeros((maxTime, size))
    hy = np.zeros((maxTime, size))


    for qTime in range(0, maxTime, 1):
        for mm in range(0, size-1, 1):
            hy[qTime][mm] = hy[qTime-1][mm] + (ez[qTime-1][mm + 1] - ez[qTime-1][mm]) / imp0

        for mm in range(1, size, 1):
            ez[qTime][mm] = ez[qTime-1][mm] + (hy[qTime][mm] - hy[qTime][mm - 1]) * imp0

        ez[qTime][0] = np.exp(-(qTime-30)*(qTime-30)/100)

    return hy, ez

---
---
---

In [39]:
import matplotlib.animation as animation
from IPython.display import HTML 

def anim_1D(x, u):

    frame_count = u.shape[0]

    fig, ax = plt.subplots()
    line, = ax.plot(x, u[0,:])
    plt.ylim(np.min(u)*1.1,np.max(u)*1.1)
    plt.xlabel('x'), plt.ylabel('T')

    def init():  # only required for blitting to give a clean slate.
        line.set_ydata([np.nan] * len(x))
        return line,

    def animate(i):
        #print(i)
        line.set_ydata(u[i,:])  # update the data.
        return line,

    ani = animation.FuncAnimation(
        fig, animate, init_func=init, interval=20, blit=True, save_count=50, frames=frame_count)
    
    plt.close(ani._fig)
    
    return ani

In [46]:
# def run (size1, size2, loopSize, omega = 1, coupling = 1, loopLoss = 0, imp0 = 377.0, maxTime = 250)

## ez1, hy1, ez2, hy2, ezLoop, hyLoop

ez1, hy1, ez2, hy2, ezLoop, hyLoop = run(50,50,70)

### SOME ISSUE HERE with the scatterTerminalsRun, Doesn't match the 3.12 from problem

# Now use the function


<ipython-input-43-fea3e8971bf7>:28: ComplexWarning: Casting complex values to real discards the imaginary part
  ez1[qTime][0] = np.exp(1j*omega*maxTime)


In [47]:
ani = anim_1D(range(50), ez1)
HTML(ani.to_html5_video())

In [48]:
ani = anim_1D(range(50), hy1)
HTML(ani.to_html5_video())

In [49]:
ani = anim_1D(range(70), ezLoop)
HTML(ani.to_html5_video())

In [50]:
ani = anim_1D(range(70), hyLoop)
HTML(ani.to_html5_video())

In [51]:
ani = anim_1D(range(50), ez2)
HTML(ani.to_html5_video())

In [52]:
ani = anim_1D(range(50), hy2)
HTML(ani.to_html5_video())